In [3]:
!pip install pymorphy2
!pip install fasttext
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.6 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
     |████████████████████████████████| 6.6 MB 55.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import numpy as np
import pandas as pd
import re
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
from nltk.stem import WordNetLemmatizer
import pymorphy2
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
import transformers
import torch
import tensorflow as tf
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
#from transformers import BertTokenizer, BertModel, BertConfig

import transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import json
import gc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
!gdown --id 16PkXbMaFEZQjV_bCGgq9n_R5ZQq1V1bp
!gdown --id 16TlUyn7u7-LVGVmMW9LS0fpD6oQqmtg5
#train = pd.read_csv('../input/scan-classification-challange/df_train.csv')
train = pd.read_csv('df_train.csv')
#test = pd.read_csv('../input/scan-classification-challange/df_test.csv', index_col=0)
test = pd.read_csv('df_test.csv')
train.drop_duplicates(subset={'text'}, inplace=True) 
train.sample(3)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16PkXbMaFEZQjV_bCGgq9n_R5ZQq1V1bp
To: /content/df_train.csv
100% 16.7M/16.7M [00:00<00:00, 234MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16TlUyn7u7-LVGVmMW9LS0fpD6oQqmtg5
To: /content/df_test.csv
100% 11.0M/11.0M [00:00<00:00, 163MB/s]


,text,class
46036,"Пятерых обвиняемых - Хасана Тунджа, Айше Сегют...",Уголовное обвинение
9763,Вот цитата из телефонного разговора Марины Бар...,O
10703,Несчастный случай произошел в г. Лесной Свердл...,"Чрезвычайные происшествия, чрезвычайные ситуации"


In [7]:
# # Названия категорий переводим в числовой формат и записываем в отдельный столбец

train['labels'] = train['class'].astype('category').cat.codes
train.sample(5)

,text,class,labels
50656,"Он отметил, что был знаком с Лори более 30 лет.",O,0
18495,Об этом заявил заместитель министра внутренних...,O,0
18,"Михаил Соколов: Кстати говоря, я бы заметил, в...",O,0
35895,"Говорит, что в начале августа в холле здания Г...",O,0
38157,"В мае 2021 года суд приговорил лидера ""Левого ...",Привлечение к уголовной ответственности,32


In [9]:
# Теперь выведем все слова длиной менее 3-х символов и дополним список стоп-слов
# пробовал искать среди стопслов равных 4-м, но там получаем слова 'газа', 'прав', 'дела', 'Рост'
# как мне кажется данные слова, могут быть полезны для score

stopwords_new = set()
mas_stop = set()
for words in train['text']:
    for i in words.split():
        if len(i) <= 2:
            mas_stop.add(i)
stopwords_new = set(stopwords_ru).union(mas_stop)

In [10]:
# Создадим функцию по очистке данных. Будем переводить слова в нижний регистр, 
# удалять стоп слова, удалять числа и раздичные знаки которые не несут смысловой нагрузки. 
# Все слова преобразуем к их первоначалоной форме (Лемматизация)

morph = pymorphy2.MorphAnalyzer()
patterns = "[A-Z|a-z|0-9!#$%&'()*+,./:“″;”<=>?@[\]^_`{|}~—\"\-•–«»]+"
#stops = set(stopwords.words("russian"))
def clean(text):
    text = text.lower()
    text = re.sub(patterns, ' ', text)
    tokens = []
    for token in text.split():
        if token and token not in stopwords_new:
            token = token.strip()
            token = morph.normal_forms(token)[0]  # Лемматизация
            #token = stemmer.stem(token) # Стеммизация
            tokens.append(token)
    return ' '.join(tokens)

In [11]:
# Применим функцию очистки к train и test

train['clean'] = train['text'].apply(lambda x: clean(x))
test['clean'] = test['text'].apply(lambda x: clean(x))
train[['clean', 'text']]

,clean,text
0,продолжать кудрин лично уверенный нацпроект на...,"При этом, продолжает Кудрин, лично он уверен: ..."
1,сотрудник го отдел хранить информация работник...,Сотрудники 1-го отдела хранили информацию о ра...
2,сирийский служба безопасность пока это справля...,Сирийские службы безопасности пока с этим спра...
3,мвд сообщить днк мужчина совпасть образец найт...,"В МВД сообщили, что ДНК мужчины совпала с обра..."
4,сообщить среда пресс секретарь белый дом джена...,Об этом сообщила в среду пресс-секретарь Белог...
...,...,...
59995,представлять тяжело находиться беларусь постоя...,"Я не представляю, как тяжело вам сейчас находи..."
59996,рамка возбудить уголовный дело дать правовой о...,"""В рамках уже возбужденного уголовного дела бу..."
59997,заявить руководитель национальный центральный ...,Об этом заявил руководитель национального цент...
59998,ранее лайф рассказывать тёмный дело мушвиг абд...,Ранее Лайф рассказывал о темных делах Мушвига ...


In [ ]:
dels = ['Выемки', 'Нападение', 'Манипулирование рынком', 'Отмывание денег', 'Закрытие филиала, представительства']
#train_df = train.take(list(dels))
#train.loc[train['class'].str.startswith(["Нападение", "Выемки"])]
#searchfor = ['john', 'doe']
train_df = train[~train['class'].str.contains('|'.join(dels))]
train_df['class'].value_counts()

## FastText

In [12]:
ft_train, ft_test = train_test_split(train, random_state=42, test_size=0.2, stratify = train['class'])

In [13]:
def to_ft_label(s):
    return '__label__'+s.replace(',','_').replace(' ','_').replace('-','_')

labels_dict = {}
for g in train['class']:
    labels_dict[to_ft_label(g)] = g

In [14]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
train.iloc[:, 0] = train.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
test.iloc[:, 1] = test.iloc[:, 1].apply(lambda x: ' '.join(simple_preprocess(x)))

In [15]:
col = ['class', 'text']

# train
train_for_ft = ft_train[col]
train_for_ft['class']=[to_ft_label(s) for s in train_for_ft['class']]

# test
test_for_ft = ft_test[col]
test_for_ft['class']=[to_ft_label(s) for s in test_for_ft['class']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [16]:
train_for_ft.to_csv('train_for_ft.csv', index=False, sep=' ', header=False, escapechar=" ")
test_for_ft.to_csv('test_for_ft.csv', index=False, sep=' ', header=False, escapechar=" ")

In [20]:
import fasttext
model = fasttext.train_supervised('train_for_ft.csv', lr = 0.9)

In [21]:
model.test('test_for_ft.csv')

(11881, 0.8876357208989142, 0.8876357208989142)

In [22]:
def predict(test):
    return labels_dict[ model.predict(test['text'], k=1)[0][0] ]
test['predictions'] = test.apply(predict,axis=1)

test.head()

,Unnamed: 0,text,class,clean,predictions
0,0,куйбышевский районный суд петербурга заключил ...,O,куйбышевский районный суд петербург заключить ...,O
1,1,результате разлива нефти более тысяч баррелей ...,O,результат разлив нефть тысяча баррель сырьё по...,Утечка нефти
2,2,бы сказал самым популярным популярность котора...,O,сказать самый популярным… популярность который...,O
3,3,это стандартная практика хотя данном случае эк...,O,это стандартный практика хотя данный случай эк...,O
4,4,сам павел своих товарищах по делу сказал что в...,O,павел свой товарищ дело сказать заметно поседеть,O


In [23]:
submission = pd.DataFrame({'id':range(0, len(test)),
                           'class':test['predictions'].values},
                          columns=['id', 'class'])
submission.to_csv('submission1.csv', index=False)
submission.head()

,id,class
0,0,O
1,1,Утечка нефти
2,2,O
3,3,O
4,4,O


## Model sklearn
Logistic Regression

In [25]:
# Выделим X, y. X - это будет, наш обработанный текст, y -  наш класс

y = train.labels.values
X = train.drop(['class', 'text'], axis=1)

In [ ]:
#y = train.encoded_cat.values
#X = train.drop(['encoded_cat', 'text'], axis=1)

In [26]:
## Разделим все данные на train test в соотношении 80/20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y)

Необходимо преобразовать тест в токены, сделаем несколькими способами и посмотрим на результат

In [27]:
## CountVectorizer преобразует текст в матрицу количества токенов

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

X_train_review_bow = vect.fit_transform(X_train['clean'])
X_test_review_bow = vect.transform(X_test['clean'])
X_sub_rev_bow = vect.transform(test['clean'])

print('X_train_review_bow shape: ', X_train_review_bow.shape)
print('X_test_review_bow shape: ', X_test_review_bow.shape)
print('X_sub_review_bow shape: ', X_sub_rev_bow.shape)

X_train_review_bow shape:  (47521, 40067)
X_test_review_bow shape:  (11881, 40067)
X_sub_review_bow shape:  (40000, 40067)


In [28]:
## Tf-Idf преобразует текст в матрицу функций TF-IDF (частота обратная частоте документа)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_review_tfidf = vectorizer.fit_transform(X_train['clean'])
X_test_review_tfidf = vectorizer.transform(X_test['clean'])
X_sub_review_tfidf = vectorizer.transform(test['clean'])

print('X_train_review_tfidf shape: ', X_train_review_tfidf.shape)
print('X_test_review_tfidf shape: ', X_test_review_tfidf.shape)
print('X_sub_review_tfidf shape: ', X_sub_review_tfidf.shape)

X_train_review_tfidf shape:  (47521, 40067)
X_test_review_tfidf shape:  (11881, 40067)
X_sub_review_tfidf shape:  (40000, 40067)


In [29]:
# Построим модель логистической регресии, в качестве метрики будем выводить accuracy и F1

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', random_state=42, max_iter=1000)
clf.fit(X_train_review_tfidf, y_train)

#y_pred = clf.predict(X_test_review_tfidf)
y_pred = clf.predict(X_test_review_tfidf)
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

Test Accuracy:  0.8872990489016076
Test F1:  0.8724557378339716


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [31]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', random_state=42, max_iter=1000)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)

print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

Test F1:  0.9097792179480577


Попробуем другие модели sklearn

In [32]:
# C-классификация опорных векторов

from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train_review_bow, y_train)
y_pred_svc = svclassifier.predict(X_test_review_bow)

print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

Test F1:  0.9097792179480577


In [ ]:
# Классификатор, реализующий голосование k ближайших соседей

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train_review_bow, y_train)
y_pred = knn.predict(X_test_review_bow)

print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

In [34]:
from sklearn import model_selection, preprocessing, metrics, linear_model, svm
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline

In [35]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    classifier.fit(feature_vector_train, label)

    predictions = classifier.predict(feature_vector_valid)    

    return metrics.f1_score(y_test, predictions, average='weighted')

In [41]:
# Random Over-sampling: случайным образом дублируются некоторые элементы из миноритарного класса.
# Random Over Sampling

from collections import Counter
ros = RandomOverSampler(random_state=777)
ros_xtrain_tfidf, ros_train_y = ros.fit_resample(X_train_review_tfidf, y_train)
#print('Resampled dataset shape %s' % Counter(ros_train_y))
accuracyROS = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_xtrain_tfidf, ros_train_y, X_test_review_tfidf)
print ("Logistic regression ROS, WordLevel TFIDF: ", accuracyROS)

Logistic regression ROS, WordLevel TFIDF:  0.8991637315885191


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [46]:
ros = RandomOverSampler(random_state=777)
ros_xtrain_bow, ros_train_y = ros.fit_resample(X_train_review_bow, y_train)
#print('Resampled dataset shape %s' % Counter(ros_train_y))
accuracyROS = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_xtrain_bow, ros_train_y, X_test_review_bow)
print ("Logistic regression ROS, WordLevel BOW: ", accuracyROS)

Logistic regression ROS, WordLevel BOW:  0.9015411346007323


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [42]:
sm = SMOTE(k_neighbors=1)
sm_xtrain_tfidf, sm_train_y = sm.fit_resample(X_train_review_tfidf, y_train)
accuracySMOTE = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),sm_xtrain_tfidf, sm_train_y, X_test_review_tfidf)
print ("Logistic regression SMOTE, WordLevel TFIDF: ", accuracySMOTE)

Logistic regression SMOTE, WordLevel TFIDF:  0.9007812919692348


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Гипер параметры

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

"""model = LogisticRegression(multi_class='multinomial', penalty='l2', random_state=42)
#penalty = ['l1','l2']
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
grid = dict(alpha=alpha)
c_values = [100, 10, 1.0]
#param_grid = {'C':[1, 10]}
param_grid = dict(C=c_values, solver=solver)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, verbose=10, cv=cv, scoring='accuracy', error_score=0)
grid_search.fit(X_train_review_bow, y_train)
#print("Best: %f using %s" % (grid_search.best_params_))"""

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve

clf = LogisticRegression(multi_class='multinomial', solver='newton-cg', penalty='l2', C = 10, random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)

print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))
#prob_true, prob_pred = calibration_curve(y_test, y_pred, n_bins=5)
#print(prob_true)
#print(prob_pred)

Test F1:  0.9073548376632803


In [45]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='saga', penalty='l2', C = 100, random_state=42, max_iter=1000)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)

print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

Test F1:  0.9071339575992307


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


Стекинг

In [49]:
from sklearn.ensemble import StackingClassifier
#from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

#data, target = load_breast_cancer(return_X_y=True)

estimators = [('lr', LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', random_state=42, max_iter=1000)), ('svc', SVC(kernel='linear'))]
modelClf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

#X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.3, random_state=12)

modelClf.fit(X_train_review_bow, y_train)
y_pred = clf.predict(X_test_review_bow)

print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))
#print(modelClf.score(X_valid, y_valid))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  UserWarning,


Test Accuracy:  0.910192744718458
Test F1:  0.9071339575992307


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## Bert TenzorFlow

In [ ]:
!pip install transformers
!pip install pytorch
!pip install pytorch-transformers

In [ ]:
import transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import json
import gc

## Distill-Bert fine-tuning - Huggingface and Pytorch

In [ ]:
!pip install datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from datasets import Dataset
from sklearn.model_selection import train_test_split

## DistilBert Freeze

In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
#train.drop([['class', 'encoded_cat']], axis=1, inplace=True)
train.sample(3)

In [ ]:
# Применяем get_dumies для таргета
train_class = pd.get_dummies(train, columns=['class'])
train_class.sample(3)

In [ ]:
#train['labels'] = train_class['encoded_cat'].values.tolist()
train

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 2e-05

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
#tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
#rubert = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
bert = AutoModel.from_pretrained("cointegrated/rubert-tiny")
#distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [ ]:
from torch import nn
#from transformers import BertModel
from torch.utils.data import Dataset, DataLoader

class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True, # Устарел
            #padding= 'max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Создаем dataset и dataloader для нейронной сети

train_size = 0.8
train_data=train.sample(frac=train_size,random_state=200)
test_data=train.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
# замораживаем слои

#for param in distilbert.parameters():
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
# solved
#sum(param.numel() for param in bert.parameters() if param.requires_grad)

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# Создаем Class для нашей модели
# Здесь я использовал 2 линейных слоя поверх модели DistilBERT с блоком отсева и ReLu в качестве функции активации. 
# num_classes будет количеством классов, доступных в вашем наборе данных. Модель вернет логит-оценки для каждого класса

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.bert = bert
        #self.distilbert = distilbert
        #self.l1 = AutoModel.from_pretrained("cointegrated/rubert-tiny")
        self.pre_classifier = torch.nn.Linear(312, 312)
        self.dropout = torch.nn.Dropout(0.1)
        
        self.relu = nn.ReLU()
        #self.pre_classifier = torch.nn.Linear(312, 312)
        self.classifier = torch.nn.Linear(312, 50)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = bert_output.last_hidden_state[:, 0, :] # hidden_state = output_1[0]
        #pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(last_hidden_state)
        pooler = self.relu(pooler)
        #pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

In [ ]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.AdamW(params =  model.parameters(), eps=1e-8, lr=LEARNING_RATE)

In [ ]:
from transformers import get_linear_schedule_with_warmup

#epochs = 4
#total_steps = len(training_loader) * EPOCHS
# Create the learning rate scheduler.
#scheduler = get_linear_schedule_with_warmup(optimizer, 
                                         #   num_warmup_steps = 0, # Default value in run_glue.py
                                          #  num_training_steps = total_steps)

In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))
f1_m = f1_score(targets, final_outputs, average='weighted', zero_division = 1)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
def train(epoch):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        acc = val_hamming_loss(outputs, targets)
        
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)
    #train_loss, train_acc = train(epoch)
    #valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    #print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%') #' Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))
f1_m = f1_score(targets, final_outputs, average='weighted', zero_division = 1)
print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")
print(f'F_1 = {f1_m}')

In [ ]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.bert = bert
        #self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        
        self.dropout = torch.nn.Dropout(0.2)
        self.rnn = nn.GRU(312,
                        256,
                        num_layers=2,
                        batch_first=True,
                        dropout=0.1)
        #self.relu =  nn.ReLU()
        #self.pre_classifier = torch.nn.Linear(768, 768)
        self.classifier = torch.nn.Linear(312, 50)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        #pooler = self.pre_classifier(pooler)
        #pooler = self.relu(pooler)
        #pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.AdamW(params =  model.parameters(), eps=1e-8, lr=LEARNING_RATE)

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

In [ ]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))
f1_m = f1_score(targets, final_outputs, average='weighted', zero_division = 1)
print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")
print(f'F_1 = {f1_m}')

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

MAX_LEN = 390
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 1e-05

#tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
rubert = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
#bert = AutoModel.from_pretrained("cointegrated/rubert-tiny")
#distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.bert = rubert
        #self.distilbert = distilbert
        #self.l1 = AutoModel.from_pretrained("cointegrated/rubert-tiny")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        
        self.relu = nn.ReLU()
        #self.pre_classifier = torch.nn.Linear(312, 312)
        self.classifier = torch.nn.Linear(768, 50)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = bert_output.last_hidden_state[:, 0, :] # hidden_state = output_1[0]
        #pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(last_hidden_state)
        pooler = self.relu(pooler)
        #pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.AdamW(params =  model.parameters(), eps=1e-8, lr=LEARNING_RATE)

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

In [ ]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))
f1_m = f1_score(targets, final_outputs, average='weighted', zero_division = 1)
print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")
print(f'F_1 = {f1_m}')

In [ ]:
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

_____________________________________________________________________________________

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
bert = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
bert.cuda();
bert.eval();

def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
print(paraphrase('Каждый охотник желает знать, где сидит фазан.'))

In [ ]:
!git clone https://github.com/blanchefort/rutau.git
!pip install -r requirements.txt

In [ ]:
!pip install nlpaug

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import nlpaug.augmenter.word as naw

text = 'Россиянка Мария Бутина, судимая в США по обвинению в участии в заговоре с целью ведения деятельности в пользу иностранного государства, может стать фигурантом еще одного дела. Об этом сообщает Daily Beast. Издание обратило внимание на запрос федеральной прокуратуры в Вашингтоне, который опубликовал в Twitter журналист Спенсер Хсу. Данная бумага находилась в закрытых судебных документах. Прокуратура просит разрешение транспортировать Бутину из тюрьмы для дачи показаний по вероятному уголовному расследованию, детали которого не уточняются. Перевозку россиянки планируется осуществить тайно, в том числе и в целях безопасности самой ответчицы. 13 декабря Мария Бутина в суде призналась в работе иностранным агентом под руководством российского государственного чиновника без согласования с Генпрокуратурой США. Она пошла на сделку со следствием и признала вину по некоторым пунктам обвинения. Следующее слушание по делу назначено на 12 февраля. Бутина была задержана в США 15 июля. Американская сторона заявляет, что она пыталась повлиять на главных спонсоров Республиканской партии для продвижения российских интересов. Кремль считает все обвинения со стороны США в адрес россиянки необоснованными, а в МИД заявили, что ее подвергали пыткам в тюрьме.'
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-ru-en',
    to_model_name='facebook/wmt19-en-ru')
back_translation_aug.augment(text)